In [1]:
import torch
from torch import nn
import numpy as np

In [2]:
import typing
from typing import List, Tuple, Dict
from tqdm import tqdm
from seqeval import metrics

# Config

In [3]:
experiment_name = "glove_clipped"
embedding_path = "data/embeddings/base/clipped.glove.6B.300d.txt"
embedding_dim = 300
nn_hidden_size = 50
model_output_path = f"models/classifier_{experiment_name}.pt"

epochs = 5
batch_size = 32
learning_rate = 1e-3

train_datapath = "data/datasets/conll_2003/train.txt"
validation_datapath = "data/datasets/conll_2003/valid.txt"
test_datapath = "data/datasets/conll_2003/test.txt"

pad_tag = "<PAD>"
unk_tag = "<UNK>"

In [4]:
gpu = torch.cuda.is_available()
device = torch.device("cuda" if gpu else "cpu")

# Load Embeddings

In [5]:
words = []
vectors = []
with open(embedding_path, "r", encoding="utf-8") as fp:
    for line in fp:
        line = line.split()
        word = line[0]
        vector = np.asarray(line[1:], dtype='float32')
        words.append(word)
        vectors.append(vector)
vectors = np.asarray(vectors)

Create an embedding for both \<PAD> (all 0s) and \<UNK> (average of all embeddings) tags.

In [6]:
unk_embedding = np.mean(vectors, axis=0).reshape(1, -1)
pad_embedding = np.zeros((1, vectors.shape[1]))

In [7]:
vectors = torch.as_tensor(np.concatenate((vectors, pad_embedding, unk_embedding)))

# Load Data & Preprocess

In [8]:
def load_data_from_file(file_path: str) -> List[List[str]]:
    sentences = []
    sentence = []
    with open(file_path, "r", encoding="utf-8") as fp:
        for line in fp:
            if "-DOCSTART-" in line: # Start of new doc
                if len(sentence) > 0:
                    sentences.append(sentence)
                    sentence = []
            elif len(line) == 1: # Empty line
                if len(sentence) > 0:
                    sentences.append(sentence)
                    sentence = []
            else:
                word = line.split()
                sentence.append(word)
    if len(sentence) > 0:
        sentences.append(sentence)
    return sentences

In [9]:
def get_words_and_tags(
    data: List[List[str]], tag_index: int, lowercase: bool=True
) -> Tuple[List[List[str]], List[List[str]]]:
    """
    tag_index: [(0,POS), (1,Syntactic Chunks), (2, NER)]
    """
    sentences = []
    tags = []
    for sentence in data:
        x = []
        y = []
        for item in sentence:
            if lowercase:
                item[0] = item[0].lower()
            x.append(item[0])
            y.append(item[tag_index+1])
        sentences.append(x)
        tags.append(y)
    return sentences, tags

Load Data & Split into Words + Tags

In [10]:
train = load_data_from_file(train_datapath)
val = load_data_from_file(validation_datapath)
test = load_data_from_file(test_datapath)

In [11]:
len(train), len(val), len(test)

(14041, 3250, 3453)

In [12]:
x_train, y_train = get_words_and_tags(train, tag_index=2)
x_val, y_val = get_words_and_tags(val, tag_index=2)
x_test, y_test = get_words_and_tags(test, tag_index=2)

Compute Max Length (for padding; we compute this only from training data)

In [13]:
# We increase the max length here to account for potentially longer samples in val/test
max_length = round(max([len(sample) for sample in x_train]) * 1.3)

In [14]:
max_length

147

Set up dictionaries for converting tags to indices, tokens to indices and vice-versa.

In [15]:
token2index = {word: i for i, word in enumerate(words)}
pad_token_index = len(token2index)
unk_token_index = len(token2index) + 1
token2index[pad_tag] = pad_token_index
token2index[unk_tag] = unk_token_index

index2token = {i: word for word, i in token2index.items()}

In [16]:
tags = set([xx for x in y_train for xx in x])
tag2index = {tag: i for i, tag in enumerate(tags)}
index2tag = {i: tag for tag, i in tag2index.items()}

unk_tag_index = tag2index["O"]
pad_tag_index = tag2index["O"]

Prepare sentences for the model

In [17]:
def sequence_to_tensor(
    sequence: List[str], sequence_to_index: Dict[str, int], vocab: List[str], unk: int, pad_to: int, padding_token: int
) -> torch.Tensor:
    indices = [sequence_to_index[word] if word in vocab else unk for word in sequence]
    padding = [padding_token] * (pad_to - len(indices))
    return indices + padding

In [18]:
def tensor_to_sequence(
    tensor: torch.Tensor, index_to_sequence: Dict[int, str]
) -> List[str]:
    return [index_to_sequence[int(i)] for i in tensor]

In [19]:
x_train = torch.as_tensor([sequence_to_tensor(sentence, token2index, words, unk_token_index, max_length, pad_token_index) 
                           for sentence in x_train])

y_train = torch.as_tensor([sequence_to_tensor(sentence_tags, tag2index, tags, unk_tag_index, max_length, pad_tag_index) 
                           for sentence_tags in y_train])

In [20]:
x_val = torch.as_tensor([sequence_to_tensor(sentence, token2index, words, unk_token_index, max_length, pad_token_index) 
                         for sentence in x_val])

y_val = torch.as_tensor([sequence_to_tensor(tags, tag2index, tags, unk_tag_index, max_length, pad_tag_index) 
                         for tags in y_val])

In [21]:
x_test = torch.as_tensor([sequence_to_tensor(sentence, token2index, words, unk_token_index, max_length, pad_token_index) 
                          for sentence in x_test])

y_test = torch.as_tensor([sequence_to_tensor(tags, tag2index, tags, unk_tag_index, max_length, pad_tag_index) 
                          for tags in y_test])

In [22]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

In [23]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu, 
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu, 
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=gpu, 
)

# Model Time

In [24]:
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
class BiLSTM(nn.Module):
    def __init__(self, embeddings, embedding_dim, hidden_dim, tagset_size):
        super(BiLSTM, self).__init__()
        self.word_embeddings = nn.Embedding.from_pretrained(embeddings, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.tag_classifier = nn.Linear(hidden_dim * 2, tagset_size) #Since BiLSTM

    def forward(self, sentence):
        # (batch_size, seq_len) -> (batch_size, seq_len, embedding_dim)
        embedded = self.word_embeddings(sentence)
        # (batch_size, seq_len, embedding_dim) -> (batch_size, seq_len, hidden_dim) 
        lstm_out, _ = self.lstm(embedded)
        # (batch_size, seq_len, hidden_dim) -> (batch_size, seq_len, tagset_size)
        tag_space = self.tag_classifier(lstm_out)
        return tag_space

In [25]:
model = BiLSTM(vectors, embedding_dim, nn_hidden_size, len(tag2index))
model.double() # Since our embeddings are 32-dimensional
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training

In [26]:
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    
    # Training Loop
    for iteration, batch in enumerate(tqdm(train_dataloader)):
        # Move data to device
        sentences, tags = batch
        sentences = sentences.to(device)
        tags = tags.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(sentences)
        
        # Calculate loss
        batch_loss = loss_function(predictions.transpose(1, 2), tags)
        
        # Backward pass
        batch_loss.backward()
        optimizer.step()
        
        # Update train loss
        train_loss += batch_loss.item()
    
    # Validation Loop
    with torch.no_grad():
        for iteration, batch in enumerate(tqdm(val_dataloader)):
            # Move data to device
            sentences, tags = batch
            sentences = sentences.to(device)
            tags = tags.to(device)

            # Forward pass
            predictions = model(sentences)

            # Calculate loss
            batch_loss = loss_function(predictions.transpose(1, 2), tags)

            # Update train loss
            val_loss += batch_loss.item()
    
    # Compute the average losses for this epoch
    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    
    
    # Print Metrics
    print(
        f"Epoch: {epoch+1}/{epochs}, Train Loss = {train_loss}, \
        Validation Loss = {val_loss}"
    )

  0%|                                                                                          | 0/439 [00:00<?, ?it/s]

Epoch: 1/5, Train Loss = 0.1604013345418729,         Validation Loss = 0.0365037158817609


  0%|                                                                                          | 0/439 [00:00<?, ?it/s]

Epoch: 2/5, Train Loss = 0.02656687750011962,         Validation Loss = 0.023897807389759843


  0%|                                                                                          | 0/439 [00:00<?, ?it/s]

Epoch: 3/5, Train Loss = 0.01848523525706136,         Validation Loss = 0.018650044987627793


  0%|                                                                                          | 0/439 [00:00<?, ?it/s]

Epoch: 4/5, Train Loss = 0.014003924061038642,         Validation Loss = 0.01535610073247981


 35%|████████████████████████████▏                                                   | 155/439 [00:43<01:19,  3.59it/s]


KeyboardInterrupt: 

# Evaluation

In [ ]:
model.eval()

In [ ]:
# Test Loop
with torch.no_grad():
    y_true = []
    y_pred = []
    for iteration, batch in enumerate(tqdm(test_dataloader)):
        # Move data to device
        sentences, tags = batch
        sentences = sentences.to(device)

        # Forward pass
        predictions = model(sentences).detach().numpy().argmax(axis=2)
        
        y_true.extend(tags.detach().numpy())
        y_pred.extend(predictions)
y_true = [tensor_to_sequence(x, index2tag) for x in np.asarray(y_true)]
y_pred = [tensor_to_sequence(x, index2tag) for x in np.asarray(y_pred)]

In [ ]:
print(metrics.classification_report(y_true, y_pred, digits=2))

# Save

In [ ]:
torch.save(model.state_dict(), model_output_path)